In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard
# from data import trainGenerator, testGenerator, saveResult, testGenerator2
import tensorflow.keras.backend as K
import os, cv2
import numpy as np
import skimage.io as io
import skimage.transform as trans
import tensorflow as tf
import shutil
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.models import Model

In [7]:
import os
import shutil
# Directory containing the training data
source_directory = '/kaggle/input/deepglobe-road-extraction-dataset/train'   

# Directory where PNG images will be moved
satelliteDir = "satellite_images/"
maskedDir = "masked_images/" 

# Create the subdirectory if it doesn't exist
if not os.path.exists(satelliteDir):
    os.makedirs(satelliteDir)
    
if not os.path.exists(maskedDir):
    os.makedirs(maskedDir)

# List all files in the directory
all_files = os.listdir(source_directory)

# Filter and move PNG files to the subdirectory
for file in all_files:
    if file.endswith('.png'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(maskedDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)
    elif file.endswith('.jpg'):
        # Full path of the source file
        source_path = os.path.join(source_directory, file)
        
        # Full path of the destination file
        destination_path = os.path.join(satelliteDir, file)
        
        # Move the file to the subdirectory
        shutil.copy(source_path, destination_path)

In [8]:
sourceDir_sat="satellite_images/"
sourceDir_mask="masked_images/"
testDir_sat="testData/sat/"
testDir_mask="testData/mask/"
if not os.path.exists(testDir_sat):
    os.makedirs(testDir_sat)

if not os.path.exists(testDir_mask):
    os.makedirs(testDir_mask)
    
count=0
#train data (satellite)
files = os.listdir(sourceDir_sat)
files.sort()
for filename in files:
    if(count<100): 
        if filename.lower().endswith('.jpg'):
            source_file = os.path.join(sourceDir_sat, filename)
            destination_file = os.path.join(testDir_sat, filename)

            # Move the file
            shutil.move(source_file, destination_file)
            count+=1

count=0
#train data (masked)
files = os.listdir(sourceDir_mask)
files.sort()
for filename in files:
    if(count<100):
        if filename.lower().endswith('.png'):
            source_file = os.path.join(sourceDir_mask, filename)
            destination_file = os.path.join(testDir_mask, filename)

            # Move the file
            shutil.move(source_file, destination_file)
            count+=1

In [9]:
sourceDir_sat="satellite_images/"
sourceDir_mask="masked_images/"
trainDir_sat="trainData/sat/"
valDir_sat="valData/sat/"
trainDir_mask="trainData/mask/"
valDir_mask="valData/mask/"
#taking 600 images to train
if not os.path.exists(trainDir_sat):
    os.makedirs(trainDir_sat)

if not os.path.exists(valDir_sat):
    os.makedirs(valDir_sat)

if not os.path.exists(trainDir_mask):
    os.makedirs(trainDir_mask)

if not os.path.exists(valDir_mask):
    os.makedirs(valDir_mask)

count=0
#val data (mask)
files = os.listdir(sourceDir_mask)
files.sort()
for filename in files:
    if(count<600):
        destination_file = os.path.join(valDir_mask, filename)
    else:
        destination_file = os.path.join(trainDir_mask, filename)
    source_file = os.path.join(sourceDir_mask, filename)
    # Move the file
    shutil.move(source_file, destination_file)
    count+=1

count=0
#train data (satellite)
files = os.listdir(sourceDir_sat)
files.sort()
for filename in files:
    if(count<600):
        destination_file = os.path.join(valDir_sat, filename)
    else:
        destination_file = os.path.join(trainDir_sat, filename)
    source_file = os.path.join(sourceDir_sat, filename)
    # Move the file
    shutil.move(source_file, destination_file)
    count+=1

In [12]:
# Code to crop images to specific size

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def crop_image(src, save_path):
    TEST_SET = os.listdir(src)
    TEST_SET.sort()
    img_h = 256
    img_w = 256
    stride = img_h-40
    count = 0
    for n in range(len(TEST_SET)):
        image_name = TEST_SET[n]
        path1 = image_name[0:-7]+'mask.png'  #rename mask
        # load the image
        # image = cv2.imread(os.path.join(src,image_name), cv2.IMREAD_UNCHANGED)
        # image = cv2.imread(os.path.join(src,image_name))
        print(os.path.join(src,image_name))
        image = io.imread(os.path.join(src,image_name))
        
        print(image.shape)
        h, w, _ = image.shape
        # h, w = image.shape

        num = 0;
        #image = img_to_array(image)
        # padding_img = (padding_img - np.min(padding_img)) / (np.max(padding_img) - np.min(padding_img))

        print('[{}/{}], croping:{}'.format(n+1, len(TEST_SET), image_name))

        #mask_whole = np.zeros((h, w, 1), dtype=np.uint8)
        #temp = np.zeros((img_h, img_h), dtype=np.uint8)

        for i in range(0, (h // stride)+1):
            for j in range(0, (w // stride)+1):
                h_begin = i * stride
                w_begin = j * stride
                
                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)
                
                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)
                
                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w] 
                if num <= 9:
                    path1 = image_name[0:-4]+'0'+ str(num)+'.jpg'
                    #path1 = image_name[0:-4]+'0'+ str(num)+'.png'
                else:
                    path1 = image_name[0:-4]+str(num)+'.jpg'
                    #path1 = image_name[0:-4]+str(num)+'.png'
                
                #io.imsave(save_path + path1, crop)
                cv2.imwrite(save_path + path1, crop)
                num = num + 1
        print('Done!')

In [13]:
# Cropping Train Images
trainDir_sat="trainData/sat/"
valDir_sat="valData/sat/"
trainDir_mask="trainData/mask/"
valDir_mask="valData/mask/"

cropsatDir_train = "trainData/cropped_sat/"
cropmaskDir_train = "trainData/cropped_mask/" 

cropsatDir_val = "valData/cropped_sat/"
cropmaskDir_val = "valData/cropped_mask/" 

if not os.path.exists(cropsatDir_train):
    os.makedirs(cropsatDir_train)
    
if not os.path.exists(cropmaskDir_train):
    os.makedirs(cropmaskDir_train)

if not os.path.exists(cropsatDir_val):
    os.makedirs(cropsatDir_val)
    
if not os.path.exists(cropmaskDir_val):
    os.makedirs(cropmaskDir_val)

crop_image(trainDir_sat, cropsatDir_train)
#crop_image(trainDir_mask, cropmaskDir_train)
crop_image(valDir_sat, cropsatDir_val)
#crop_image(valDir_mask, cropmaskDir_val)

trainData/sat/203439_sat.jpg
(1024, 1024, 3)
[1/5526], croping:203439_sat.jpg
Done!
trainData/sat/203471_sat.jpg
(1024, 1024, 3)
[2/5526], croping:203471_sat.jpg
Done!
trainData/sat/203508_sat.jpg
(1024, 1024, 3)
[3/5526], croping:203508_sat.jpg
Done!
trainData/sat/203675_sat.jpg
(1024, 1024, 3)
[4/5526], croping:203675_sat.jpg
Done!
trainData/sat/204189_sat.jpg
(1024, 1024, 3)
[5/5526], croping:204189_sat.jpg
Done!
trainData/sat/20451_sat.jpg
(1024, 1024, 3)
[6/5526], croping:20451_sat.jpg
Done!
trainData/sat/204584_sat.jpg
(1024, 1024, 3)
[7/5526], croping:204584_sat.jpg
Done!
trainData/sat/204654_sat.jpg
(1024, 1024, 3)
[8/5526], croping:204654_sat.jpg
Done!
trainData/sat/204713_sat.jpg
(1024, 1024, 3)
[9/5526], croping:204713_sat.jpg
Done!
trainData/sat/205035_sat.jpg
(1024, 1024, 3)
[10/5526], croping:205035_sat.jpg
Done!
trainData/sat/205045_sat.jpg
(1024, 1024, 3)
[11/5526], croping:205045_sat.jpg
Done!
trainData/sat/20537_sat.jpg
(1024, 1024, 3)
[12/5526], croping:20537_sat.jpg

In [14]:
from tqdm import tqdm

In [15]:
def encoder_block(inputs, num_filters): 
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3,padding = 'same')(inputs) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 

    # Max Pooling with 2x2 filter 
    x = tf.keras.layers.MaxPool2D(pool_size = (2, 2), strides = 2)(x) 
    return x

In [16]:
def decoder_block(inputs, skip_features, num_filters): 
    # Upsampling with 2x2 filter 
    x = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides = 2, padding = 'same')(inputs) 
      
    # Copy and crop the skip features  
    # to match the shape of the upsampled input 
    skip_features = tf.image.resize(skip_features, size = (x.shape[1], x.shape[2])) 
    x = tf.keras.layers.Concatenate()([x, skip_features]) 
      
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
  
    # Convolution with 3x3 filter followed by ReLU activation 
    x = tf.keras.layers.Conv2D(num_filters, 3, padding = 'same')(x) 
    x = tf.keras.layers.Activation('relu')(x) 
      
    return x

In [17]:
def IoU(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    #y_true_f = np.array(K.flatten(y_true))
    #y_pred_f = np.array(K.flatten(y_pred))
    #y_pred_f[y_pred_f >= 0.5] = 1
    #y_pred_f[y_pred_f < 0.5] = 0
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection) / (K.sum(y_true_f) + K.sum(y_pred_f))

def final_loss(y_true, y_pred):
    loss1 = binary_crossentropy(y_true, y_pred)
    loss2 = 1 - IoU(y_true, y_pred)
    return loss1 + loss2
def unet(pretrained_weights = None,input_size = (256,256,3)):
    
    inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv4)
    #drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv5)
    #drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(learning_rate = 2e-4), loss = final_loss, metrics = [IoU])
    #model.compile(optimizer = SGD(lr=1e-3, decay=0.0, momentum=0.9, nesterov=True), loss = final_loss, metrics = [IoU])
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

In [22]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator


# def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict,image_color_mode = "rgb",
#                     mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
#                     flag_multi_class = False,num_class = 2,save_to_dir = None,target_size = (256,256),seed = 1):
#     '''
#     can generate image and mask at the same time
#     use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
#     if you want to visualize the results of generator, set save_to_dir = "your path"
#     '''
#     image_datagen = ImageDataGenerator(**aug_dict)
#     mask_datagen = ImageDataGenerator(**aug_dict)
#     image_generator = image_datagen.flow_from_directory(
#         train_path,
#         classes = [image_folder],
#         class_mode = None,
#         color_mode = image_color_mode,
#         target_size = target_size,
#         batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = image_save_prefix,
#         seed = seed)
#     mask_generator = mask_datagen.flow_from_directory(
#         train_path,
#         classes = [mask_folder],
#         class_mode = None,
#         color_mode = mask_color_mode,
#         target_size = target_size,
#         batch_size = batch_size,
#         save_to_dir = save_to_dir,
#         save_prefix  = mask_save_prefix,
#         seed = seed)
#     train_generator = zip(image_generator, mask_generator)
#     for (img,mask) in train_generator:
#         #img,mask = adjustData(img,mask,flag_multi_class,num_class)
#         img = img / 255.0
#         mask = mask / 255.0
#         yield (img,mask)


def trainGenerator(batch_size, train_path, image_folder, mask_folder, aug_dict, image_color_mode="rgb",
                    mask_color_mode="grayscale", image_save_prefix="image", mask_save_prefix="mask",
                    flag_multi_class=False, num_class=2, save_to_dir=None, target_size=(256, 256), seed=1):
    '''
    Can generate image and mask at the same time.
    Uses the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same.
    If you want to visualize the results of generator, set save_to_dir = "your path".
    '''
    image_datagen = ImageDataGenerator(**aug_dict)
    mask_datagen = ImageDataGenerator(**aug_dict)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes=[image_folder],
        class_mode=None,
        color_mode=image_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=image_save_prefix,
        seed=seed
    )
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes=[mask_folder],
        class_mode=None,
        color_mode=mask_color_mode,
        target_size=target_size,
        batch_size=batch_size,
        save_to_dir=save_to_dir,
        save_prefix=mask_save_prefix,
        seed=seed
    )
    
    # Check if generators produce the same number of samples
    if image_generator.batch_size != mask_generator.batch_size:
        raise ValueError("Batch sizes of image_generator and mask_generator must be equal.")

    train_generator = zip(image_generator, mask_generator)
    
    for (img, mask) in train_generator:
        if img.shape[0] != mask.shape[0]:
            raise ValueError("Batch sizes of images and masks do not match.")
        
        # Normalize images and masks
        img = img / 255.0
        mask = mask / 255.0
        
        yield (img, mask)


In [23]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping


In [24]:
data_gen_args = dict(rotation_range=90.,
                    #width_shift_range=0.1,
                    #height_shift_range=0.1,
                    #shear_range=0.1,
                    #zoom_range=0.1,
                    fill_mode='nearest',
                    horizontal_flip=True,
                    vertical_flip=True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss', factor=0.2, patience=3, verbose=0, mode='min', epsilon=1e-4, 
                              cooldown=0, min_lr=1e-6)
visual = TensorBoard(log_dir='./VanillaUNet_log', histogram_freq=0, write_graph=True, write_images=True)
earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=0, mode='min')

train_Gene = trainGenerator(1,'trainData/','cropped_sat','cropped_mask',data_gen_args,save_to_dir = None)
# to be fixed(train-val split)
val_Gene = trainGenerator(1,'valData/','cropped_sat','cropped_mask',data_gen_args)

In [25]:
model = unet()

In [26]:
model_checkpoint = ModelCheckpoint('U-Net.keras', monitor='val_loss',verbose=1, save_best_only=True)
model.fit(train_Gene,steps_per_epoch=100,epochs=200,
                    callbacks=[model_checkpoint, visual, reduce_lr, earlystop], 
                    validation_data=val_Gene, validation_steps=220)#step_per_epoch and validation_steps equals to number of samples divide batchsize

Found 138150 images belonging to 1 classes.
Found 138150 images belonging to 1 classes.
Epoch 1/200


I0000 00:00:1726179943.467012     340 service.cc:145] XLA service 0x7e9d14004450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726179943.467052     340 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1726179943.467056     340 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1726179961.822209     340 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - io_u: 0.0338 - loss: 1.5297Found 15000 images belonging to 1 classes.
Found 15000 images belonging to 1 classes.

Epoch 1: val_loss improved from inf to 1.18614, saving model to U-Net.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 154ms/step - io_u: 0.0338 - loss: 1.5282 - val_io_u: 0.0321 - val_loss: 1.1861 - learning_rate: 2.0000e-04
Epoch 2/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - io_u: 0.0360 - loss: 1.2308
Epoch 2: val_loss did not improve from 1.18614
100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 125ms/step - io_u: 0.0360 - loss: 1.2305 - val_io_u: 0.0378 - val_loss: 1.1970 - learning_rate: 2.0000e-04
Epoch 3/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - io_u: 0.0304 - loss: 1.1822
Epoch 3: val_loss improved from 1.18614 to 1.17883, saving model to U-Net.keras
100/100 ━━━━━━━━━━━━━━━━━━━━ 13s 132ms/step - io_u: 0.0304 - loss: 1.1823 - val_io_u: 0.0248 - val_loss: 1.1788 - learning_rate: 2.0000e-04
Epoch 4/200
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/st

In [72]:
! rm -rf /kaggle/working/predicted

In [73]:
def predict_z(src, predict_path):
    TEST_SET = os.listdir(src)
    model = unet()
    print('Loading Model weights...')
    model.load_weights('U-Net.keras')
    print('Completed!')
    img_h = 256
    img_w = 256
    stride = img_h - 16
    for n in range(len(TEST_SET)):
        path = TEST_SET[n]
        path1 = path[0:-7] + 'mask.png'  # Rename mask
        image = io.imread(os.path.join(src, path))
        h, w, _ = image.shape

        image = image / 255.0
        print('[{}/{}], predicting: {}'.format(n + 1, len(TEST_SET), path))

        mask_whole = np.zeros((h, w, 1), dtype=np.uint8)

        for i in range(0, (h // stride) + 1):
            for j in range(0, (w // stride) + 1):
                h_begin = i * stride
                w_begin = j * stride

                if h_begin + img_h > h:
                    h_begin = h_begin - (h_begin + img_h - h)

                if w_begin + img_w > w:
                    w_begin = w_begin - (w_begin + img_w - w)

                crop = image[h_begin:h_begin + img_h, w_begin:w_begin + img_w, :3]
                ch, cw, _ = crop.shape

                if ch != img_h or cw != img_w:
                    print('Invalid size!')
                    print(i, j, h_begin, w_begin, ch, cw)
                    continue

                crop = np.expand_dims(crop, axis=0)
                pred = model.predict(crop, verbose=2)
                pred = pred.reshape((img_h, img_w, 1)).astype(np.float64)

                # Debugging
                print('Prediction min:', np.min(pred))
                print('Prediction max:', np.max(pred))

                # Thresholding
                pred[pred >= 0.25] = 1
                pred[pred < 0.25] = 0
                pred = pred * 255

                # Update mask
                mask_whole[h_begin:h_begin + img_h, w_begin:w_begin + img_w] = pred

        # Ensure correct shape and dtype
        mask_whole = np.squeeze(mask_whole)
        mask_whole = mask_whole.astype(np.uint8)
        
        # Save mask
        cv2.imwrite(predict_path + path1, mask_whole[0:h, 0:w])
        print('Done!')


In [74]:
fileDir = "/kaggle/working/testData/sat"  #test images(1024*1024)
preDir = "predicted/" #Dir of predict mask
if not os.path.exists(preDir):
    os.makedirs(preDir)
predict_z(fileDir, preDir)  

Loading Model weights...
Completed!
[1/100], predicting: 106737_sat.jpg
1/1 - 1s - 830ms/step
Prediction min: 0.015695344656705856
Prediction max: 0.4937213361263275
1/1 - 0s - 18ms/step
Prediction min: 0.022603260353207588
Prediction max: 0.47682204842567444
1/1 - 0s - 18ms/step
Prediction min: 0.07028178870677948
Prediction max: 0.29754066467285156
1/1 - 0s - 18ms/step
Prediction min: 0.01068133395165205
Prediction max: 0.4937213361263275
1/1 - 0s - 18ms/step
Prediction min: 0.010681330226361752
Prediction max: 0.4937213361263275
1/1 - 0s - 18ms/step
Prediction min: 0.008414151147007942
Prediction max: 0.4937213361263275
1/1 - 0s - 17ms/step
Prediction min: 0.008098547346889973
Prediction max: 0.4937213361263275
1/1 - 0s - 17ms/step
Prediction min: 0.0574578233063221
Prediction max: 0.31068485975265503
1/1 - 0s - 17ms/step
Prediction min: 0.013181455433368683
Prediction max: 0.4937213361263275
1/1 - 0s - 17ms/step
Prediction min: 0.013181591406464577
Prediction max: 0.493721336126327

In [75]:
import os
import cv2
import numpy as np
import skimage.io as io

#path1 = "data/membrane/test/mask_8bit"
path1 = "/kaggle/working/testData/mask" #Dir of Ground Truth
#path1 = "data/membrane/test/mask_8bit"
#path2 = "data/membrane/test/sub_test/predict1"
path2 = "/kaggle/working/predicted" #Dir of predict map
#path2 = "data/membrane/train/predict"
sample1 = os.listdir(path1)
Iou = []#Iou for each test images
TP = 0
FP = 0
FN = 0
sum_fenmu = 0
for name in sample1:
    mask1 = io.imread(os.path.join(path1, name))
    mask1 = mask1 / 255
    #print(mask1.shape)
    mask1 = mask1.flatten()
    
    #name1 = name[0:-8]+'sat.jpg'
    #mask2 = io.imread(os.path.join(path2, name1))
    mask2 = io.imread(os.path.join(path2, name))
    mask2 = mask2 / 255.0
    #print(mask2.shape)
    #mask2[mask2 >= 0.5] = 1
    #mask2[mask2 < 0.5] = 0
    mask2 = np.stack((mask2,)*3, axis=-1)
    #print(mask2.shape)
    mask2 = mask2.flatten()
    
    tp = np.dot(mask1, mask2)
    TP = TP + tp
    fp = mask2.sum()-tp
    FP = FP + fp
    fn = mask1.sum()-tp
    FN = FN + fn
    #fenmu = mask1.sum()+mask2.sum()-tp
    fenmu = mask1.sum()+mask2.sum()-tp
    sum_fenmu = sum_fenmu + fenmu
    #element_wise = np.multiply(mask1, mask2)
    Iou.append(tp / fenmu)
    #if(tp / fenmu == 0.0):
        #print(name)
    
#print("IOU score:",Iou)
print("IOU score:",(TP / sum_fenmu))#active IoU
print("Recall: ",(TP / (TP+FN)))#recall
print("Precision: ",(TP / (TP+FP)))#precision

IOU score: 0.07882731372887296
Recall:  0.3674740062021522
Precision:  0.09120195534505683
